<a href="https://colab.research.google.com/github/desmond-rn/ComputationalLogic/blob/prolexa-plus/Assignment%20-%20Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prolexa Assignment Report 
This notebook serves are a short report for this 👉 [coursework assignment](https://github.com/desmond-rn/ComputationalLogic/blob/prolexa-plus/assignment.md). It goes thought our implementation of **Negation**, **Default rule**, and **Existential quantification** while showcasing selected examples. In our implementations, we mostly added (instead of editing) to the preexisting grammar and engine. For this demo, we will be using Prolexa Plus for its ease of use in a notebook environment. For a more in-depth introduction to Prolexa and Prolexa Plus, see the [Prolexa Plus Demo Notebook](https://github.com/desmond-rn/ComputationalLogic/blob/prolexa-plus/Prolexa_Plus_Demo_Notebook.ipynb).

## Setup

In this section, we setup the invironnment for our demo. *These might take a while to run* !

Install SWI-Prolog.

In [1]:
!apt-get install swi-prolog -qqq > /dev/null

Extracting templates from packages: 100%


Install Prolexa Plus from the branch in our repository addressing Negation.

In [2]:
%%capture
!yes | pip install git+https://github.com/desmond-rn/ComputationalLogic/@neg -qqq > /dev/null

Instantiate Prolexa Plus.


In [3]:
## For not displaying the output
%%capture   

from pyswip import Prolog
import prolexa.meta_grammar as meta

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

Define the `respond()` utility function.

In [4]:
## @title Prolexa Interact

input = 'tell me all you know'  #@param {type:"string"}

def respond(input):
  first_answer = meta.standardised_query(pl, input)[0]['Output']
  if isinstance(first_answer, bytes):
    print("'"+str(first_answer, "utf-8")+"'")
  else:
    print("'"+first_answer+"'")

respond(input)

'I know nothing'


## Negation

This section covers our implementation of Negation in Prolexa. Let's illustrate our method on the following example:
> Every teacher is happy. Donald is not happy. Therefore, Donald is not a teacher.

First, let's add adequate rules to Prolexa. 

In [5]:
respond('forget all you know')

'I am a blank slate'


In [6]:
input = 'every teacher is happy'  #@param {type:"string"}
respond(input)

'I will remember that every teacher is happy'


In [7]:
input = 'Donald is not happy'  #@param {type:"string"}
respond(input)

'I will remember that Donald is not happy'


In [8]:
respond('tell me everything you know')

'every teacher is happy. donald is not happy'


Now let's test our implementation; and the way Prolexa get's to the result.

In [9]:
input = 'who is not a teacher'  #@param {type:"string"}
respond(input)

'donald is not a teacher'


In [10]:
input = 'explain why donald is not a teacher'  #@param {type:"string"}
respond(input)

'donald is not happy; every teacher is happy; therefore donald is not a teacher'


Below, we can see that the same result is not obtained for other proper nouns initially in our grammar. This shows that Default Rule is not (yet) implemented in Prolexa's reasoning. 

In [11]:
input = 'explain why peter is not a teacher'  #@param {type:"string"}
respond(input)

'Sorry, I don't think this is the case'


### Changes to the grammar
In order for Prolexa to understand the sentences above, we made a few changes to her grammar (see `prolexa_grammar.pl`). Specifically, we added:
- the predicate `not` as seen in the lectures:
```
:- op(900, fy, not).
```

- the terms `donald`, `happy`, and `teacher` as proper noun, adjective, and noun respectively.

- additional clauses for `sentence1` and `verb_phrase` to parse negative sentences like 'Donald is not happy' in singular and plural form, by means of Prolog goals incorporated into DCG: 
```
sentence1(C) --> proper_noun(N,X),verb_phrase(N,T), {T=[(not X=>L)], C=[(not L:-true )]}.
verb_phrase(s,N) --> [is,not],property(s,M), {N=[(not M)]}.
verb_phrase(p,N) --> [are,not],property(p,M), {N=[(not M)]}.
```
- additional clause for negative questions:
```
question1(not P) --> [who],verb_phrase(s,Q),{Q=[(not _X=>P)]}.
```

### Changes to the engine

Next, we made changes to `prolexa_engine.pl`. Essentially, we reasoned by **contraposition**. Let P and Q be two propositions: proving that `P:-Q` is equivalent to proving that `not(Q):-not(P)`. This was achieved in Prolexa by adding the following clause to the existing `prove_rb` meta-interpreter:
```prolog
prove_rb(not B,Rulebase,P0,P):-
    find_clause((A:-B),Rule,Rulebase),
  prove_rb(not A,Rulebase,[p(not B,Rule)|P0],P).
```

## Defalut Rule

The goal here is to solve problems in the form:
> Most birds fly except penguins. Tweety is a bird. Therefore, assuming Tweety is not a penguin, Tweety flies.

Let's reinstall and launch Prolexa Plus from the branch in our GitHub repository that contains our Default Rule implementation :

In [12]:
%%capture   

!yes | pip uninstall prolexa -qqq > /dev/null
!yes | pip install git+https://github.com/desmond-rn/ComputationalLogic/@default-rule -qqq > /dev/null

# from pyswip import Prolog
# import prolexa.meta_grammar as meta

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

In [13]:
respond('forget all you know')

'I am a blank slate'


In [14]:
input = 'all birds fly except penguins'  #@param {type:"string"}
respond(input)

'I will remember that all birds fly except penguins'


In [15]:
input = 'Tweety is a bird'  #@param {type:"string"}
respond(input)

'I will remember that Tweety is a bird'


In [16]:
respond('tell me everything you know')

'all birds fly except penguins. tweety is a bird'


Now let's test our implementation.

In [17]:
input = 'who flies'  #@param {type:"string"}
respond(input)

'tweety flies'


In [18]:
# meta.reset_grammar()
input = 'explain why tweety flies'  #@param {type:"string"}
respond(input)

'tweety is a bird; all birds fly except penguins; therefore tweety flies'


Testing our implementation further, we see that when Tweety becomes part of the exceptions (i.e. penguins), the default rule doesn't apply anymore.

In [19]:
input = 'Tweety is a penguin'  #@param {type:"string"}
respond(input)

'I will remember that Tweety is a penguin'


In [20]:
respond('tell me everything you know')

'all birds fly except penguins. tweety is a bird. tweety is a penguin'


In [21]:
input = 'explain why tweety flies'  #@param {type:"string"}
respond(input)

'Sorry, I don't think this is the case'


### Changes to the grammar

Our approach to implementing Default Rule was based on our solution for Negation. All the changes made to the grammar and the engine for Negation remain. New additions to the grammar that are specific to Default Rule are:

- A new clause for `sentence1` to parse sentences containing the word `except`:
```
sentence1(C) --> determiner(N,M1,M2,_D),
                    noun(N,M1),
                    verb_phrase(N,M2), 
                    [except], 
                    noun(N,M3), 
                    {C=[(C1:-C2,not C3)], 
                          M1=(_X2=>C2), 
                          M2=(_X1=>C1), 
                          M3=(_X3=>C3)}.
```
- A few clauses for `verb_phrase`:
```
verb_phrase(s,N) --> [does, not],iverb(p,M), {N=[(not M)]}.
verb_phrase(p,N) --> [do, not],iverb(p,M), {N=[(not M)]}.
```

### Changes to the engine

As for the engine, we added a few rules to the `prove-rb` meta-interpreter. These two rules are largely based on the definition of `not` as implemented for 'Negation as Failure' in [chapter 3 of Simply-Logical](https://too.simply-logical.space/src/text/1_part_i/3.3.html).
```
prove_rb(not A,Rulebase,P0,P):-
    prove_rb(A,Rulebase,P0,P), !, fail.
prove_rb(not _A,_Rulebase,P,P):-!.
``` 

In conclusion for this section, we can see that Default Rule is a strong addition to Prolexa, but with consequences. The example below shows that Prolexa can now reach conclusions she is unable to prove.

In [22]:
input = 'explain why peter does not fly'  #@param {type:"string"}
respond(input)

'therefore peter does not fly'


## Existential quantification

The goal in this final section is to solve problems in the form:
> Some humans are geniuses. Geniuses win prizes. Therefore, some humans win prizes.

The reasoning implemented in the previous sections is available here. Let's reinstall and launch the appropriate (default) branch from our GitHub repository:

In [23]:
%%capture   

!yes | pip uninstall prolexa -qqq > /dev/null
!yes | pip install git+https://github.com/desmond-rn/ComputationalLogic/ -qqq > /dev/null

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

In [24]:
respond('forget all you know')

'I am a blank slate'


In [25]:
input = 'Some humans are geniuses'  #@param {type:"string"}
respond(input)

'I will remember that Some humans are geniuses'


In [26]:
input = 'Geniuses win prizes'  #@param {type:"string"}
respond(input)

'I will remember that Geniuses win prizes'


In [27]:
respond('tell me everything you know')

'some humans are geniuses. every genius wins prizes'


Now, let's test our implementation.

In [28]:
input = 'who wins prizes'  #@param {type:"string"}
respond(input)

'some humans win prizes'


In [29]:
# meta.reset_grammar()
input = 'explain why some humans win prizes'  #@param {type:"string"}
respond(input)

'some humans are geniuses; some humans are geniuses; every genius wins prizes; therefore some humans win prizes'


Notice that Prolexa Plus explains twice that 'some humans are geniuses'. This is due to our
implementation based on a list of clauses (see changes to the engine below). Fixing this (post-processing) issues could be part of some future work.



### Changes to the grammar

The new additions to the grammar rules are:

- New predicates for 'genius' and 'win' declaring them as noun and (intransitive) verb respectively:
```
pred(genius,   1,[n/genius]).
pred(win,     1,[v/win]).
```

- A new clause for `verb_phrase` in order to treat `win` as a transitive verb. This cheat declares `win` as an intransitive verb followed necessarily by the word 'prizes'. This was an ad hoc solution in order to run this demo as expected:
```
verb_phrase(N,M) --> iverb(N,M),[prizes].
```

- A new clause for `determiner` giving 'some' a meaning that can be written as a list of two clauses:
```
determiner(p, sk=>H1, sk=>H2, [(H2:-true),(H1:-true)]) -->[some].
```



### Changes to the engine

Since we decided to treat the meaning of the determiner 'some' as a **list of two clauses**, this required extensive additions to the Prolexa engine:

- A new `prove_question` clause:
```
prove_question(Query,SessionId,Answer):-
	findall(R,prolexa:stored_rule(SessionId,R),Rulebase),
	( prove_rb([Query,SecondQuery],Rulebase) ->
		transform([Query,SecondQuery],Clauses),
		phrase(sentence(Clauses),AnswerAtomList),
		atomics_to_string(AnswerAtomList," ",Answer)
	; Answer = 'Sorry, I don\'t think this is the case'
	).	
```

- Similarly, a new clause for `explain_question` needed for proofs:
```
explain_question([Query,SecondQuery],SessionId,Answer):-
	findall(R,prolexa:stored_rule(SessionId,R),Rulebase),
	( prove_rb([Query,SecondQuery],Rulebase,[],Proof) ->
		maplist(pstep2message,Proof,Msg),
		phrase(sentence1([(Query:-true),(SecondQuery:-true)]),L),
		atomic_list_concat([therefore|L]," ",Last),
		append(Msg,[Last],Messages),
		atomic_list_concat(Messages,"; ",Answer)
	; Answer = 'Sorry, I don\'t think this is the case'
	).
```

- New clauses for `prove_rb`:
```
prove_rb([true,true],_Rulebase,P,P) :- !.
% prove_rb([A,true],Rulebase,P0,P) :- prove_rb(A,Rulebase,P0,P), !.
% prove_rb([true,A],Rulebase,P0,P) :- prove_rb(A,Rulebase,P0,P), !.
prove_rb([A,C],Rulebase,P0,P):-
	find_clause([(A:-B),D],Rule,Rulebase),
	(
		var(D) ->  		%%% D is uninstantiated
		prove_rb([B,C],Rulebase,[p(A,Rule)|P0],P) 
		; 
		D = (C:-E),
		prove_rb([B,E],Rulebase,[p(A,Rule),p(C,Rule)|P0],P)
	).
```

- Another set of clauses needed for proofs:
```
prove_rb([A,C],Rulebase,P0,P):-
	prove_rb((A,C),Rulebase,P0,P).
```

- At the heart of our approach, we devised a new strategy for finding body of clauses whose heads are known and unifiable; this with lists:
```
find_clause(Clause,Rule,[Rule|_Rules]):-
	Rule = [Rule1|Rule2],
	(
		Clause = [Clause1,Clause2] ->
		(
			copy_term([Rule1],[Clause1]) ->
			( Rule2 = [El|_] -> Clause2 = El ; true )
			; copy_term(Rule2,[Clause1]),
			Clause2 = Rule1
		)
		;
		(
			copy_term([Rule1],[Clause]) ->
			true
		; 	copy_term(Rule2,[Clause])
		)
	).
find_clause(Clause,Rule,[_Rule|Rules]):-
	find_clause(Clause,Rule,Rules).
```

- Finally, a new clause to turn literals in lists into clauses by adding `true` to their body:
```
transform([A,B],[(A:-true),(B:-true)]).
```

## Conclusion

This notebook showcases three important resoning strategies we implemented in Prolexa. Though we highlighted most of our work, we left out minor changes we made to the `meta_grammar.py` file. We essentially stopped it from extending the grammar because it was hindering our implemention. 

Negation, Default Rule and Existential Quantification are all interesting additions to Prolexa's reasoning capabilities. However, at the moment, most of these reasoning techniques do not function together. For example, Prolexa is unable to parse the following sentence:
> Some humans are _not_ happy.

Combining these techniques will be part of our future work, along with implementing disjunction and abduction. The other part of the work will attempt to make our implementation with Prolexa Plus' extended grammar work as well as it does with Prolexa.